In [3]:
import cv2
import os
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [4]:
def load_data(path, label):
    data = []
    labels = []
    for img_file in os.listdir(path):
        img_path = os.path.join(path, img_file)
        try:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print("Error loading image:", img_path)
                continue

            img = cv2.resize(img, (100, 100))  # Resize the image to a fixed size
            data.append(img)
            labels.append(label)
        except Exception as e:
            print("Error loading image:", img_path, "Error:", str(e))
            continue
            
    return data, labels

# Load and preprocess the data
male_path = "/home/db/Projects/AI/Datasets/FP/Male"
female_path = "/home/db/Projects/AI/Datasets/FP/Female"

male_data, male_labels = load_data(male_path, "male")
female_data, female_labels = load_data(female_path, "female")

# Combine male and female data
all_data = male_data + female_data
all_labels = male_labels + female_labels

# Convert data and labels to numpy arrays
all_data = np.array(all_data)
all_labels = np.array(all_labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_data, all_labels, test_size=0.2, random_state=42)

# Convert labels to numeric values
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Convert the numeric labels to one-hot encoded binary class matrices
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build the gender prediction neural network model
gender_model = Sequential()
gender_model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(100, 100, 1)))
gender_model.add(MaxPooling2D(pool_size=(2, 2)))
gender_model.add(Flatten())
gender_model.add(Dense(64, activation="relu"))
gender_model.add(Dense(2, activation="softmax"))

# Compile the model
gender_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Train the model
gender_model.fit(X_train, y_train, epochs=20, batch_size=32)

# Evaluate the model
accuracy = gender_model.evaluate(X_test, y_test)[1]
print("Accuracy:", "{:.2f}%".format(accuracy*100))

# Save Model
gender_model.save('gender_model.h5')

/home/db/Projects/AI/fingerprint_recognition/fp_env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 114s 695ms/step - accuracy: 0.7152 - loss: 139.0468
Epoch 2/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 101s 671ms/step - accuracy: 0.7979 - loss: 0.6177
Epoch 3/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 143s 677ms/step - accuracy: 0.7851 - loss: 0.5788
Epoch 4/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 141s 673ms/step - accuracy: 0.7874 - loss: 0.5505
Epoch 5/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 90s 600ms/step - accuracy: 0.8033 - loss: 0.5212
Epoch 6/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 55s 370ms/step - accuracy: 0.7977 - loss: 0.5161
Epoch 7/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 63s 423ms/step - accuracy: 0.7916 - loss: 0.5169
Epoch 8/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 115s 762ms/step - accuracy: 0.7927 - loss: 0.5130
Epoch 9/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 108s 716ms/step - accuracy: 0.7936 - loss: 0.5105
Epoch 10/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 112s 746ms/step - accuracy: 0.7905 - loss: 0.5137
Epoch 11/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 115s 763ms/step - accuracy: 0.7936 - loss: 0.5095
Epoch 12/

Accuracy: 80.33%


In [5]:
# Function to load and preprocess fingerprint images for handedness prediction
def load_handedness_data(path, handedness):
    data = []
    labels = []
    for img_file in os.listdir(path):
        img = cv2.imread(os.path.join(path, img_file), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (100, 100))  # Resize the image to a fixed size
        data.append(img)
        labels.append(handedness)
    return data, labels

# Load and preprocess the data for handedness prediction
right_path = "/home/db/Projects/AI/Datasets/FP/Right"
left_path = "/home/db/Projects/AI/Datasets/FP/Left"

right_data, right_labels = load_handedness_data(right_path, "right")
left_data, left_labels = load_handedness_data(left_path, "left")

# Combine right and left-handed data
all_data_handedness = right_data + left_data
all_labels_handedness = right_labels + left_labels

# Convert data and labels to numpy arrays
all_data_handedness = np.array(all_data_handedness)
all_labels_handedness = np.array(all_labels_handedness)

# Split the data into training and testing sets for handedness prediction
X_train_handedness, X_test_handedness, y_train_handedness, y_test_handedness = train_test_split(
    all_data_handedness, all_labels_handedness, test_size=0.2, random_state=42
)

# Convert handedness labels to numeric values
le = LabelEncoder()
y_train_handedness = le.fit_transform(y_train_handedness)
y_test_handedness = le.transform(y_test_handedness)

# Convert the numeric labels to one-hot encoded binary class matrices
y_train_handedness = to_categorical(y_train_handedness)
y_test_handedness = to_categorical(y_test_handedness)

# Build the handedness prediction neural network model
handedness_model = Sequential()
handedness_model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(100, 100, 1)))
handedness_model.add(MaxPooling2D(pool_size=(2, 2)))
handedness_model.add(Flatten())
handedness_model.add(Dense(64, activation="relu"))
handedness_model.add(Dense(2, activation="softmax"))

# Compile the model
handedness_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Train the model
handedness_model.fit(X_train_handedness, y_train_handedness, epochs=20, batch_size=32)

# Evaluate the model
accuracy_handedness = handedness_model.evaluate(X_test_handedness, y_test_handedness)[1]
print("Handedness Accuracy:", "{:.2f}%".format(accuracy_handedness*100))

# Save Model
handedness_model.save('handedness_model.h5')

/home/db/Projects/AI/fingerprint_recognition/fp_env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 123s 755ms/step - accuracy: 0.4881 - loss: 185.6513
Epoch 2/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 40s 264ms/step - accuracy: 0.5128 - loss: 0.6930
Epoch 3/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 40s 256ms/step - accuracy: 0.5002 - loss: 0.6932
Epoch 4/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 41s 255ms/step - accuracy: 0.5046 - loss: 0.6932
Epoch 5/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 42s 265ms/step - accuracy: 0.4947 - loss: 0.6932
Epoch 6/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 40s 265ms/step - accuracy: 0.4990 - loss: 0.6932
Epoch 7/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 39s 261ms/step - accuracy: 0.4857 - loss: 0.6933
Epoch 8/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 40s 267ms/step - accuracy: 0.5049 - loss: 0.6931
Epoch 9/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 40s 267ms/step - accuracy: 0.4853 - loss: 0.6933
Epoch 10/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 40s 264ms/step - accuracy: 0.5158 - loss: 0.6929
Epoch 11/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 41s 264ms/step - accuracy: 0.4939 - loss: 0.6932
Epoch 12/20
150/

Handedness Accuracy: 48.92%


In [6]:
# Function to load and preprocess fingerprint images for finger type prediction
def load_finger_type_data(path, finger_type):
    data = []
    labels = []
    for img_file in os.listdir(path):
        img = cv2.imread(os.path.join(path, img_file), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (100, 100))  # Resize the image to a fixed size
        data.append(img)
        labels.append(finger_type)
    return data, labels

# Load and preprocess the data for finger type prediction
thumb_path = "/home/db/Projects/AI/Datasets/FP/Thumb"
index_path = "/home/db/Projects/AI/Datasets/FP/Index"
middle_path = "/home/db/Projects/AI/Datasets/FP/Middle"
ring_path = "/home/db/Projects/AI/Datasets/FP/Ring"
little_path = "/home/db/Projects/AI/Datasets/FP/Little"

thumb_data, thumb_labels = load_finger_type_data(thumb_path, "thumb")
index_data, index_labels = load_finger_type_data(index_path, "index")
middle_data, middle_labels = load_finger_type_data(middle_path, "middle")
ring_data, ring_labels = load_finger_type_data(ring_path, "ring")
little_data, little_labels = load_finger_type_data(little_path, "little")

# Combine finger type data
all_data_finger_type = thumb_data + index_data + middle_data + ring_data + little_data
all_labels_finger_type = thumb_labels + index_labels + middle_labels + ring_labels + little_labels

# Convert data and labels to numpy arrays
all_data_finger_type = np.array(all_data_finger_type)
all_labels_finger_type = np.array(all_labels_finger_type)

# Split the data into training and testing sets for finger type prediction
X_train_finger_type, X_test_finger_type, y_train_finger_type, y_test_finger_type = train_test_split(
    all_data_finger_type, all_labels_finger_type, test_size=0.2, random_state=42
)

# Convert finger_type labels to numeric values
le = LabelEncoder()
y_train_finger_type = le.fit_transform(y_train_finger_type)
y_test_finger_type = le.transform(y_test_finger_type)

# Convert the numeric labels to one-hot encoded binary class matrices
y_train_finger_type = to_categorical(y_train_finger_type)
y_test_finger_type = to_categorical(y_test_finger_type)

# Build the finger type prediction neural network model
finger_type_model = Sequential()
finger_type_model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(100, 100, 1)))
finger_type_model.add(MaxPooling2D(pool_size=(2, 2)))
finger_type_model.add(Flatten())
finger_type_model.add(Dense(64, activation="relu"))
finger_type_model.add(Dense(5, activation="softmax"))

# Compile the model
finger_type_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Train the model
finger_type_model.fit(X_train_finger_type, y_train_finger_type, epochs=20, batch_size=32)

# Evaluate the model
accuracy_finger_type = finger_type_model.evaluate(X_test_finger_type, y_test_finger_type)[1]
print("Finger Type Accuracy:", "{:.2f}%".format(accuracy_finger_type*100))

# Save Model
finger_type_model.save('finger_type_model.h5')

/home/db/Projects/AI/fingerprint_recognition/fp_env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 26s 166ms/step - accuracy: 0.1953 - loss: 355.0583
Epoch 2/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 161ms/step - accuracy: 0.1942 - loss: 1.7069
Epoch 3/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 161ms/step - accuracy: 0.2123 - loss: 1.6070
Epoch 4/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.2066 - loss: 1.6037
Epoch 5/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.2310 - loss: 1.5876
Epoch 6/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 161ms/step - accuracy: 0.2381 - loss: 1.5753
Epoch 7/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 161ms/step - accuracy: 0.2416 - loss: 1.5581
Epoch 8/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.2334 - loss: 1.5723
Epoch 9/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 158ms/step - accuracy: 0.2075 - loss: 1.6085
Epoch 10/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 161ms/step - accuracy: 0.2244 - loss: 1.5891
Epoch 11/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 161ms/step - accuracy: 0.2338 - loss: 1.5626
Epoch 12/20
150/1

Finger Type Accuracy: 26.75%


In [8]:
# Function to preprocess
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (100, 100))  # Resize the image to a fixed size
    img = img.reshape(1, 100, 100, 1)  # Add batch dimension
    img = img.astype('float32') / 255.0  # Normalize the image
    return img

# Function to predict the gender of the fingerprint
def predict_gender(image_path, gender_model):
    img = preprocess_image(image_path)
    gender_prediction = gender_model.predict(img)
    gender_label = "Male" if gender_prediction[0][0] >= 0.5 else "Female"
    return gender_label

# Function to predict the handedness of the fingerprint
def predict_handedness(image_path, handedness_model):
    img = preprocess_image(image_path)
    handedness_prediction = handedness_model.predict(img)
    handedness_label = "Right" if handedness_prediction[0][0] >= 0.5 else "Left"
    return handedness_label

# Function to predict the finger type of the fingerprint
def predict_finger_type(image_path, finger_type_model):
    img = preprocess_image(image_path)
    finger_type_prediction = finger_type_model.predict(img)
    finger_types = {0: "Thumb", 1: "Index", 2: "Middle", 3: "Ring", 4: "Little"}
    finger_type_label = finger_types[np.argmax(finger_type_prediction)]
    return finger_type_label

# Load the pre-trained models
gender_model = load_model('gender_model.h5')
handedness_model = load_model("handedness_model.h5")
finger_type_model = load_model("finger_type_model.h5")

# Example usage
image_path = "/home/db/Projects/AI/Datasets/FP/Altered/Altered-Hard/1__M_Left_index_finger_CR.BMP"
gender = predict_gender(image_path, gender_model)
handedness = predict_handedness(image_path, handedness_model)
finger_type = predict_finger_type(image_path, finger_type_model)

# Print predictions
print("Gender:", gender)
print("Handedness:", handedness)
print("Finger Type:", finger_type)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Gender: Female
Handedness: Right
Finger Type: Middle
